In [2]:
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from db import db
from utils.recommender import get_ratings_binary_matrix, get_movies_ratings_means
import numpy as np

2024-05-17 23:30:17.821807: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
ratings_matrix = db.load_sparse_ratings("cinematch_regression").toarray()

In [4]:
means = get_movies_ratings_means(ratings_matrix, get_ratings_binary_matrix(ratings_matrix))

In [5]:
intercept_lt = db.get_intercept_latent_factors("cinematch_regression")
users_lt = db.get_users_latent_factors("cinematch_regression")
movies_lt = db.get_movies_latent_factors("cinematch_regression")

In [6]:
predictions = np.matmul(movies_lt, users_lt.T) + intercept_lt
predictions += means

In [7]:
predictions.shape

(10000, 5000)

In [8]:
predictions_list = []
for i_user in range(predictions.shape[1]):
    predictions_list.append({
        "index": i_user,
        "predictions": predictions[:,i_user].tolist()
    })

In [9]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["cinematch_regression"]
collection_predictions = db["predictions"]

In [10]:
collection_predictions.insert_many(predictions_list)

InsertManyResult([ObjectId('66483d1e6aaa24253bb40067'), ObjectId('66483d1e6aaa24253bb40068'), ObjectId('66483d1e6aaa24253bb40069'), ObjectId('66483d1e6aaa24253bb4006a'), ObjectId('66483d1e6aaa24253bb4006b'), ObjectId('66483d1e6aaa24253bb4006c'), ObjectId('66483d1e6aaa24253bb4006d'), ObjectId('66483d1e6aaa24253bb4006e'), ObjectId('66483d1e6aaa24253bb4006f'), ObjectId('66483d1e6aaa24253bb40070'), ObjectId('66483d1e6aaa24253bb40071'), ObjectId('66483d1e6aaa24253bb40072'), ObjectId('66483d1e6aaa24253bb40073'), ObjectId('66483d1e6aaa24253bb40074'), ObjectId('66483d1e6aaa24253bb40075'), ObjectId('66483d1e6aaa24253bb40076'), ObjectId('66483d1e6aaa24253bb40077'), ObjectId('66483d1e6aaa24253bb40078'), ObjectId('66483d1e6aaa24253bb40079'), ObjectId('66483d1e6aaa24253bb4007a'), ObjectId('66483d1e6aaa24253bb4007b'), ObjectId('66483d1e6aaa24253bb4007c'), ObjectId('66483d1e6aaa24253bb4007d'), ObjectId('66483d1e6aaa24253bb4007e'), ObjectId('66483d1e6aaa24253bb4007f'), ObjectId('66483d1e6aaa24253bb400